# Bernoulli Verification Example - Helix

Linear elastic solution of a helix in 3d. The starting and ending points of the helix lie on the X axis. One of the ends is fully constrained, and a force is applied on the free end in X direction. The test is succesful if:

- the free end makes motion only in X direction
- on the constrained end, only a force in X direction is invoked, despite all dofs being constrained

In [70]:
from polymesh.recipes import circular_helix
fnc = circular_helix(1, 5)
fnc(1)

(0.5403023058681398, 0.8414709848078965, 5)

In [71]:
from polymesh.space import CartesianFrame
from polymesh import PolyData, LineData
import numpy as np

frame = CartesianFrame(dim=3)

L = 8.0

space = np.linspace(0, 20*np.pi, 300)
coords = np.array(list(map(circular_helix(L, 2), space)))
topo = np.zeros((coords.shape[0] - 1  , 2))
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1
topo = topo.astype(int)

mesh = PolyData(coords=coords, frame=frame)
mesh['helix'] = LineData(topo=topo, frame=frame)

mesh.plot(notebook=False)

In [72]:
coords[0], coords[-1]

(array([8., 0., 0.]),
 array([ 8.00000000e+00, -1.95943488e-14,  1.25663706e+02]))

In [73]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from sigmaepsilon.solid.fem.cells import B2 as Beam
from neumann.linalg import linspace, Vector
from polymesh.space import StandardFrame, PointCloud, frames_of_lines
from polymesh.topo.tr import L2_to_L3
from polymesh.space.utils import index_of_closest_point
  
import numpy as np
from numpy import pi as PI

xEnd = coords[-1]
if Beam.NNODE == 3:
    coords, topo = L2_to_L3(coords, topo)
iN = index_of_closest_point(coords, xEnd)

w, h = 1., 1.  # width and height of the rectangular cross section
F = 100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = w * h**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

# space
GlobalFrame = StandardFrame(dim=3)

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[iN, :3] = global_load_vector
fixity[0, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
# 1) displace the mesh
dofsol = structure.nodal_dof_solution(store='dofsol')
forces = structure.nodal_forces()
cell_forces_i = structure.internal_forces()
cell_forces_e = structure.external_forces()
reactions = structure.reaction_forces()


In [74]:
dofsol[iN], forces[iN], reactions[0]

(array([-1.69533298e+03,  7.70038245e+01,  4.56451985e+02,  3.50513013e-07,
        -2.69820624e+01,  1.66253132e+00]),
 array([  0.,   0., 100.,   0.,   0.,   0.]),
 array([-1.93562284e-07,  2.05861052e-07, -9.99999999e+01, -2.27832846e-05,
        -1.77396703e-05, -4.24137523e-06]))

In [75]:
forces[iN, 2], reactions[0, 2]

(100.0, -99.99999993797951)

In [76]:
structure.internal_forces(target='global')[-1]

array([[ 2.42956102e+01,  2.42956102e+01],
       [ 9.69719039e+01,  9.69719039e+01],
       [-2.48458984e+00, -2.48458984e+00],
       [-2.65428705e-09, -2.65428705e-09],
       [ 4.29799168e+00, -7.95807864e-09],
       [ 1.67747782e+02,  3.97903932e-09]])

In [77]:
structure.internal_forces(target=GlobalFrame)[-1]

array([[ 2.42956102e+01,  2.42956102e+01],
       [ 9.69719039e+01,  9.69719039e+01],
       [-2.48458984e+00, -2.48458984e+00],
       [-2.65428705e-09, -2.65428705e-09],
       [ 4.29799168e+00, -7.95807864e-09],
       [ 1.67747782e+02,  3.97903932e-09]])

In [78]:

structure.internal_forces(target='global')
structure.internal_forces(target='local')
structure.internal_forces(target='global', cells=[0, 5])
structure.external_forces(target='global')
structure.external_forces(target='local')
structure.external_forces(target='local', cells=[0, 5])
structure.internal_forces(target=GlobalFrame)
structure.internal_forces(target=GlobalFrame, cells=[0])


{0: array([[[ 2.42956099e+01,  2.42956099e+01]],
 
        [[ 9.69719039e+01,  9.69719039e+01]],
 
        [[ 2.48459002e+00,  2.48459002e+00]],
 
        [[ 1.58258673e-05,  1.58258673e-05]],
 
        [[-2.19497280e-07,  4.29799178e+00]],
 
        [[ 2.45205418e-05, -1.67747758e+02]]])}